# DASH app

### step 1 import modules

In [1]:
from dash import dash, dash_table, html, dcc
from dash.dependencies import Input, Output
import pandas as pd
from dash.exceptions import PreventUpdate

import dash_bootstrap_components as dbc

import plotly.express as px

### step 2 import data

In [2]:
df = pd.read_csv('./dash_pyzero/data/all_proportions_iglas.csv', sep=':')
df

,UserLanguage,Description,Option,Group,Tag,Composite,Count,Prop
0,EN,Age,18,32,General,Continuous,47,0.276471
1,EN,Age,19,32,General,Continuous,32,0.188235
2,EN,Age,20,32,General,Continuous,13,0.076471
3,EN,Age,21,32,General,Continuous,16,0.094118
4,EN,Age,22,32,General,Continuous,8,0.047059
...,...,...,...,...,...,...,...,...
1274,RU,Year of education,2 years,36,General,Yes,523,0.577901
1275,RU,Year of education,3 years,36,General,Yes,69,0.076243
1276,RU,Year of education,4 years,36,General,Yes,37,0.040884
1277,RU,Year of education,5 years,36,General,Yes,27,0.029834


### step 3 app declaration

In [3]:
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.CERULEAN], prevent_initial_callbacks=True)
app

## step 4 work on the layout of the app

https://dash.plotly.com/dash-core-components


In [4]:
app.layout = html.Div([


dbc.CardImg(
                        src="/assets/tagc.png",
                        top=True,
                        style={"width": "8rem"},
                        className="ml-3"),

dbc.CardImg(
                        src="/assets/inlab.png",
                        top=True,
                        style={"width": "8rem"},
                        className="ml-3"),

dbc.CardImg(
                        src="/assets/iglas.png",
                        top=True,
                        style={"width": "8rem"},
                        className="ml-3"),



    html.Button('Show Data', id='button'),
    html.Div(id='datatable'),


    html.H1(children='We start with exploring individual variables'),
	html.Div([
        "Select the type of variable: (by default all variables are selected)",

##################### radioitems

        dcc.RadioItems(
{
        'General': 'General',
        'GK': 'Genetic Knowledge',
        'GR': 'Genetic Rights',
        'HR': 'Human Rights'
   },
   value='General', 
        id='analytics-input')
    ]),
	html.Br(),
    html.Div(id='analytics-output'),


##################### dropdown from radio items
    html.Div([

        html.Br(),
        html.Label(['Choose variable to visualise:'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(
            id='opt-dropdown',
            value='',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Select the variable to plot...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'
            ),
            ],style={'width': '50%', 'display': 'inline-block'}
        ),

    html.Div(id='dd-output-container'),


################### graph part
html.Div([
        dcc.Graph(id='our_graph1'),
        dcc.Graph(id='our_graph2'),

    ],className='nine columns'),


])


################### ################### ################### Callbacks


@app.callback(
    Output(component_id='analytics-output', component_property='children'),
    Input(component_id='analytics-input', component_property='value')
)
def update_city_selected(input_value):
    return f'You selected: {input_value}'






@app.callback(
#datatable
    Output('datatable', 'children'),
    Input('button', 'n_clicks'),
Input(component_id='analytics-input', component_property='value')
)
def generate_upload_file(n_clicks, value):  
    if n_clicks:
        if value=='All':
            dfx = df.copy()
            data_dict = dfx.to_dict('records')
            headers_list = list(dfx.columns.values)
            headers_dict = ([{'id': _, 'name': _} for _ in headers_list])
        else:
            dfx = df[df['Tag']==value]
            data_dict = dfx.to_dict('records')
            headers_list = list(dfx.columns.values)
            headers_dict = ([{'id': _, 'name': _} for _ in headers_list])

        return dash_table.DataTable(
                data=data_dict,
                columns=headers_dict,
                filter_action="native",
                sort_action="native",
                sort_mode="multi",
                row_deletable=True,
                page_action="native",
                page_current= 0,
                page_size= 10,
            )
    else: raise PreventUpdate


##### dropdown

@app.callback(
    Output('opt-dropdown', 'options'),
    Input(component_id='analytics-input', component_property='value')
)
def update_output(value):
    dfx = df[df['Tag']==value]

    return [{'label': i, 'value': i} for i in dfx['Description'].unique()]




@app.callback(
[Output('our_graph1','figure'),Output('our_graph2','figure')], 

    [Input('opt-dropdown', 'value')]
)
def build_graph(selected_option):
    dff=df[df['Description']==selected_option].copy()

    fig1 = px.pie(dff, values="Count", names="Option", facet_col="UserLanguage", hole=.3)

    fig2 = px.bar(dff, x="Option", y="Count", facet_col="UserLanguage", title="Bar graph for proportions for variable {}".format(dff.Description.unique()))

    return fig1, fig2


if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


Don't worry. We will manage it one by one.

What is metadata? In this context, we organized variables somehow based on what they tell. The principle is the same for different data projects. It is important to know additional information about variables when we create structural models. For example, we see that responses are similar for certain type of questions. In the same way, we also understand that certain variables correlate, this could imply that these variables can be grouped together. If we don't have a preconception of what the variable structure should be like, we need to do extensive visualization to understand or make arguments for certain trends in the analysis.

Metadata in exploratory projects, like above, can also guide how the finished DASH-board will look like.

In this case, first, we organized the variables into four categories. Look at the metadata file.

In [5]:
!pwd

/home/mrinalmanu/Documents/notebooks_python


In [6]:
!head dash_pyzero/data/metadata.csv

Variable:Description:Option:Group:Composite:Tag
LE3.003:What information can be obtained from your genetic material?:Ancestry:1:No:HR
LE3.004:What information can be obtained from your genetic material?:Health:1:No:HR
LE3.005:What information can be obtained from your genetic material?:Potential to succeed in different domains aka for example sports:1:No:HR
LE3.006:What information can be obtained from your genetic material?:Personal traits aka for example extroversion:1:No:HR
LE3.007:What information can be obtained from your genetic material?:I do not know:1:No:HR
LE3.008:What information can be obtained from your genetic material?:Other:1:No:HR
LE3.009:What information can be obtained from your genetic material?:Other as Text:1:Text:HR
LE3.025:Can using genetic information affect human rights?:Yes no dontknow:2:Yes:HR
LE3.026:How relevant is genetic information to the following personal rights? :Right to security including privacy:3:No:HR


https://dash.plotly.com/dash-core-components

In [7]:
from dash import Dash, html, dcc

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal')
])

if __name__ == '__main__':
    app.run(debug=True)

In [8]:
from dash import Dash, dcc, html

app = Dash(__name__)

app.layout = html.Div([
    dcc.Slider(-5, 10, 1, value=-3)
])

if __name__ == '__main__':
    app.run(debug=True)

# packaging 101

https://docs.conda.io/projects/conda-build/en/latest/user-guide/tutorials/build-pkgs.html
https://pypi.org/project/conda-pack/

Docker

https://www.docker.com/

# Assignment

Take the metadata and create a small bar-plot application. Put TSU logo on the top. [1]

Optional (plus points)

Create a conda environment, and install dash within. The do [1] and package the environment.



EON